# 연구 실행 코드
아래의 코드 셀을 실행(Shift+Enter)하여 연구를 진행하세요.

In [ ]:

import os
from ultralytics import YOLO
from ultralytics.utils.torch_utils import model_info

# 환경 설정
base_path = r"D:\Virtual_Fahion\GraduationThesis2"
data_yaml = os.path.join(base_path, "data", "data.yaml")
save_dir = os.path.join(base_path, "saveModels")

def extract_metrics(model_list, data_path, save_path):
    print("\n🚀 [Step 7] Extracting Quantitative Metrics")
    
    collected_data = []
    
    for m_name in model_list:
        # 경로 탐색 로직 (폴더명 규칙에 따라 수정 필요)
        # 예: arch_yolov8n, baseline_yolov8n, scale_yolov8s 등 가능한 경로 확인
        possible_names = [f"arch_{m_name}", f"baseline_{m_name}", f"scale_{m_name}", f"train_{m_name}_aug_True"]
        
        weights_path = None
        for p_name in possible_names:
            temp_path = os.path.join(save_path, p_name, "weights", "best.pt")
            if os.path.exists(temp_path):
                weights_path = temp_path
                break
        
        if weights_path:
            print(f"Analyzing {m_name}...")
            model = YOLO(weights_path)
            
            # 1. 검증
            metrics = model.val(data=data_path, split='val', verbose=False)
            
            # 2. 정보 추출
            info = model_info(model.model, detailed=False, verbose=False)
            
            # 3. 속도
            latency = sum(metrics.speed.values())
            
            data = {
                "Model": m_name,
                "mAP50": round(metrics.box.map50, 4),
                "Latency(ms)": round(latency, 2),
                "Params(M)": round(info[1] / 1e6, 2),
                "GFLOPs": round(info[3], 2)
            }
            collected_data.append(data)
        else:
            print(f"⚠️ Weights not found for {m_name}")
            
    return collected_data

# 실행
if __name__ == "__main__":
    # 실험했던 모든 모델 리스트
    models_to_eval = ['yolov5n', 'yolov8n', 'yolov10n', 'yolov8s'] 
    metrics_data = extract_metrics(models_to_eval, data_yaml, save_dir)
    print("\n📊 Extracted Data:", metrics_data)
